# Part 1: Hands-On with LangChain (2025 Edition)

This notebook demonstrates LangChain's **2025 capabilities** using Azure OpenAI. We'll explore modern LCEL patterns, LangGraph agents (replacing legacy AgentExecutor), LangSmith integration, and evaluation frameworks.

## 🆕 What's New in 2025:
- **LangGraph** replaces legacy AgentExecutor patterns
- **LangSmith** production-ready monitoring and evaluation  
- **LangChain Sandbox** for safe code execution
- **Modern evaluation frameworks** integration

## Environment Setup

First, let's load our environment variables and configure our Azure OpenAI connection with **LangSmith tracing** for 2025 observability.

In [ ]:
# Environment and configuration setup with 2025 LangSmith integration
import os
import warnings
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

class NotebookConfig:
    """Configuration management for Azure OpenAI and LangSmith in Jupyter notebooks"""
    
    def __init__(self):
        # Load environment variables
        env_path = find_dotenv()
        if env_path:
            load_dotenv(env_path)
            print(f"✅ Loaded environment from: {env_path}")
        else:
            warnings.warn("No .env file found. Using system environment variables only.")
        
        self._load_azure_config()
        self._validate_config()
    
    def _load_azure_config(self):
        """Load Azure OpenAI configuration from environment variables"""
        self.azure_api_key = os.getenv('AZURE_OPENAI_API_KEY')
        self.azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
        self.azure_deployment = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'gpt-4o-mini')
        self.azure_api_version = os.getenv('AZURE_OPENAI_API_VERSION', '2024-05-01-preview')
        self.tavily_api_key = os.getenv('TAVILY_API_KEY')
    
    def _validate_config(self):
        """Validate critical configuration"""
        errors = []
        
        if not self.azure_api_key:
            errors.append("AZURE_OPENAI_API_KEY is required")
        
        if not self.azure_endpoint:
            errors.append("AZURE_OPENAI_ENDPOINT is required")
            
        if not self.tavily_api_key:
            errors.append("TAVILY_API_KEY is required for search functionality")
        
        if errors:
            raise ValueError(f"Configuration errors: {', '.join(errors)}")
        
        print("✅ All Azure OpenAI configuration validated successfully")
    
    def display_config(self):
        """Display current configuration (hiding secrets)"""
        print("Azure OpenAI Configuration:")
        print(f"  Endpoint: {self.azure_endpoint}")
        print(f"  Deployment: {self.azure_deployment}")
        print(f"  API Version: {self.azure_api_version}")
        print(f"  API Key: {'*' * 20 + self.azure_api_key[-4:] if self.azure_api_key else 'Not set'}")
        print(f"  Tavily API Key: {'*' * 20 + self.tavily_api_key[-4:] if self.tavily_api_key else 'Not set'}")

# Initialize configuration
try:
    config = NotebookConfig()
    config.display_config()
except Exception as e:
    print(f"❌ Failed to load configuration: {e}")
    print("\nPlease ensure you have a .env file with the following variables:")
    print("- AZURE_OPENAI_API_KEY")
    print("- AZURE_OPENAI_ENDPOINT")
    print("- AZURE_OPENAI_CHAT_DEPLOYMENT_NAME (optional, defaults to gpt-4o-mini)")
    print("- TAVILY_API_KEY")
    raise

### 1.1 Basic Prompting and Model I/O with LCEL

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize Azure OpenAI with configuration
llm = AzureChatOpenAI(
    azure_deployment=config.azure_deployment,
    api_version=config.azure_api_version,
    temperature=0.7,
    azure_endpoint=config.azure_endpoint,
    api_key=config.azure_api_key
)

prompt = ChatPromptTemplate.from_template("Question: {question}\nAnswer: Let's think step by step.")
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"question": "What is the capital of France?"})
print(response)

### 1.2 Sequential Processing with LCEL (Modern Pattern)

LCEL (LangChain Expression Language) is the modern way to compose operations. Let's build a sequential workflow using the pipe operator.

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Initialize Azure OpenAI with configuration
llm = AzureChatOpenAI(
    azure_deployment=config.azure_deployment,
    api_version=config.azure_api_version,
    temperature=0.7,
    azure_endpoint=config.azure_endpoint,
    api_key=config.azure_api_key
)

# Modern LCEL approach - compose operations with pipe operator
name_prompt = ChatPromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)

catchphrase_prompt = ChatPromptTemplate.from_template(
    "Write a creative catchphrase for the following company: {company_name}"
)

# Build sequential chain using LCEL composition
name_chain = name_prompt | llm | StrOutputParser()

# Create a more complex chain that passes results between steps
def create_sequential_chain():
    """Creates a sequential chain using modern LCEL patterns"""
    
    # Step 1: Generate company name
    step1 = (
        {"product": RunnablePassthrough()} 
        | name_prompt 
        | llm 
        | StrOutputParser()
    )
    
    # Step 2: Generate catchphrase using the company name
    step2 = (
        {"company_name": step1}
        | catchphrase_prompt
        | llm
        | StrOutputParser()
    )
    
    # Combine results into final output
    return {
        "company_name": step1,
        "catchphrase": step2,
        "product": RunnablePassthrough()
    }

# Execute the sequential chain
sequential_chain = create_sequential_chain()
product = "colorful, eco-friendly socks"

print(f"Input: {product}")
print("\\n🔗 Running sequential LCEL chain...")

result = sequential_chain.invoke(product)
print(f"\\n✅ Results:")
print(f"Product: {result['product']}")
print(f"Company Name: {result['company_name']}")
print(f"Catchphrase: {result['catchphrase']}")

### 1.3 Streaming with LCEL

One of LCEL's key advantages is built-in streaming support for real-time responses.

In [ ]:
import time
from langchain_core.callbacks import BaseCallbackHandler

class StreamingCallbackHandler(BaseCallbackHandler):
    """Custom callback handler to demonstrate streaming"""
    
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(token, end="", flush=True)

# Create a streaming chain
streaming_prompt = ChatPromptTemplate.from_template(
    "Write a brief story about {topic}. Make it engaging and creative."
)

streaming_chain = (
    streaming_prompt 
    | llm.with_config({"callbacks": [StreamingCallbackHandler()]})
    | StrOutputParser()
)

print("🌊 Streaming response for a story about 'space exploration':")
print("=" * 50)

# Note: In a real notebook, you'd see the text appear token by token
response = streaming_chain.invoke({"topic": "space exploration"})

print("\n" + "=" * 50)
print("✅ Streaming complete!")

### 1.4 Function Calling and Basic Agents

LangChain agents can use tools (functions) to interact with external systems. This demonstrates basic agent capabilities before we explore multi-agent systems in a dedicated notebook.

In [ ]:
# MODERN 2025 PATTERN: Using LangGraph instead of legacy AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# Initialize Azure OpenAI for agents (temperature=0 for more deterministic responses)
llm_agent = AzureChatOpenAI(
    azure_deployment=config.azure_deployment,
    api_version=config.azure_api_version,
    temperature=0,
    azure_endpoint=config.azure_endpoint,
    api_key=config.azure_api_key
)

# Define tools that the agent can use
search_tool = TavilySearchResults(
    max_results=2,
    api_key=config.tavily_api_key,
    description="Search the web for current information"
)

@tool
def calculate(expression: str) -> str:
    """A simple calculator that evaluates basic mathematical expressions.
    
    Args:
        expression: A mathematical expression like '2+2' or '4**0.5'
    """
    try:
        # Basic safety check - only allow simple math operations
        allowed_chars = set('0123456789+-*/().**')
        if not all(c in allowed_chars or c.isspace() for c in expression):
            return "Error: Only basic mathematical operations are allowed"
        
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error evaluating expression: {e}"

@tool 
def format_text(text: str, style: str = "upper") -> str:
    """Format text in different styles.
    
    Args:
        text: The text to format
        style: The formatting style ('upper', 'lower', 'title')
    """
    if style == "upper":
        return text.upper()
    elif style == "lower":
        return text.lower()
    elif style == "title":
        return text.title()
    else:
        return f"Unknown style: {style}. Use 'upper', 'lower', or 'title'"

# Create agent with tools using MODERN 2025 LangGraph pattern
tools = [search_tool, calculate, format_text]

# Modern approach: Create agent with memory using LangGraph
system_message = "You are a helpful assistant that can search the web, perform calculations, and format text. Always explain your reasoning step by step."

# Initialize memory for conversation persistence
memory = MemorySaver()

# Create the modern LangGraph agent (replaces legacy AgentExecutor)
agent = create_react_agent(
    model=llm_agent,
    tools=tools,
    prompt=system_message,
    checkpointer=memory  # 2025 feature: built-in memory
)

print("🤖 Testing MODERN 2025 LangGraph agent with multiple tool usage...")
print("=" * 50)

# Configuration for conversation threading (2025 memory feature)
config = {"configurable": {"thread_id": "demo-conversation"}}

# Test multi-step reasoning with tool usage using modern streaming
query = "Calculate 16 raised to the power of 0.5, then format the result as 'The answer is X' in title case"
print(f"🎯 Query: {query}\n")

# MODERN 2025 PATTERN: Streaming execution with memory
for step in agent.stream(
    {"messages": [("user", query)]},
    config=config,
    stream_mode="updates"
):
    if step:
        print(f"📝 Step: {step}")

print("\n" + "=" * 50)
print("✅ MODERN 2025 FEATURES DEMONSTRATED:")
print("   🔄 LangGraph agent (replaces legacy AgentExecutor)")
print("   💾 Built-in memory with conversation threading")
print("   🌊 Real-time streaming execution")
print("   🔍 LangSmith tracing (if enabled)")
print("="* 50)

## 🎯 2025 Evaluation & Monitoring Integration

Modern AI applications require comprehensive evaluation and monitoring. Let's integrate the leading 2025 frameworks.

In [ ]:
# 2025 PATTERN: Modern evaluation framework integration
from langsmith import Client
import os

# Configure LangSmith for production monitoring
if os.getenv('LANGCHAIN_API_KEY'):
    langsmith_client = Client()
    
    # Enable tracing for the agent
    os.environ['LANGCHAIN_TRACING_V2'] = 'true'
    os.environ['LANGCHAIN_PROJECT'] = 'LangChain-2025-Evaluation'
    
    print("✅ LangSmith monitoring enabled")
    print(f"📊 Project: {os.environ.get('LANGCHAIN_PROJECT')}")
else:
    print("⚠️ LangSmith not configured (add LANGCHAIN_API_KEY to enable)")

In [ ]:
# 2025 PATTERN: DeepEval integration for comprehensive testing
try:
    from deepeval import evaluate
    from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric
    from deepeval.test_case import LLMTestCase
    
    # Create evaluation metrics
    relevancy_metric = AnswerRelevancyMetric(threshold=0.7)
    faithfulness_metric = FaithfulnessMetric(threshold=0.7)
    
    # Example test case
    test_case = LLMTestCase(
        input="What are the main benefits of LangGraph?",
        actual_output="LangGraph provides state management, complex agent workflows, and streaming capabilities for multi-agent systems.",
        expected_output="LangGraph offers state management and multi-agent orchestration capabilities.",
        context=["LangGraph is LangChain's framework for building stateful, multi-agent applications"]
    )
    
    # Run evaluation
    evaluation_results = evaluate([test_case], [relevancy_metric, faithfulness_metric])
    print("✅ DeepEval integration working")
    print(f"📊 Evaluation results: {evaluation_results}")
    
except ImportError:
    print("⚠️ DeepEval not installed (pip install deepeval to enable advanced evaluation)")
except Exception as e:
    print(f"⚠️ DeepEval evaluation error: {str(e)}")

In [ ]:
# 2025 PATTERN: Cost tracking and token usage monitoring
from langchain.callbacks import get_openai_callback

def run_agent_with_cost_tracking(agent, query):
    """Run agent with comprehensive cost and performance tracking"""
    
    with get_openai_callback() as cb:
        # Configure conversation with thread ID for tracking
        config = {"configurable": {"thread_id": "cost-tracking-demo"}}
        
        # Run the agent with streaming
        response = None
        for chunk in agent.stream({"messages": [("human", query)]}, config):
            if "agent" in chunk:
                response = chunk["agent"]["messages"][-1].content
    
    # Display cost metrics
    print("\n💰 Cost Analysis:")
    print(f"  Total Tokens: {cb.total_tokens}")
    print(f"  Prompt Tokens: {cb.prompt_tokens}")
    print(f"  Completion Tokens: {cb.completion_tokens}")
    print(f"  Total Cost: ${cb.total_cost:.4f}")
    
    return response, {
        'total_tokens': cb.total_tokens,
        'cost': cb.total_cost,
        'prompt_tokens': cb.prompt_tokens,
        'completion_tokens': cb.completion_tokens
    }

# Example usage with cost tracking
if 'agent' in locals():
    test_query = "What's the current weather in San Francisco?"
    response, metrics = run_agent_with_cost_tracking(agent, test_query)
    
    print(f"\n🤖 Agent Response: {response}")
    print(f"📊 Performance Metrics: {metrics}")
else:
    print("⚠️ Agent not initialized - run the previous cells first")

## Summary: LangChain Fundamentals (2025 Edition)

This notebook covered the core concepts of LangChain using modern patterns:

### Key Concepts Learned:
1. **LCEL (LangChain Expression Language)**: Modern composition using pipe operators
2. **Sequential Processing**: Chaining operations with result passing between steps
3. **Streaming**: Real-time token streaming for better user experience  
4. **LangGraph Agents**: Modern agent framework replacing legacy AgentExecutor
5. **Evaluation Integration**: LangSmith monitoring and DeepEval testing frameworks
6. **Cost Tracking**: Production-ready token usage and cost monitoring

### 2025 LangChain Evolution:
- **LangGraph**: State management and multi-agent orchestration
- **LangSmith**: Production monitoring and evaluation workflows  
- **Modern Agent Patterns**: Memory-enabled agents with conversation threading
- **Evaluation Frameworks**: Comprehensive testing with DeepEval integration
- **Production Monitoring**: Built-in cost tracking and performance metrics

### Next Steps:
- **Multi-Agent Systems**: See `3_langchain_agents_langgraph.ipynb` for complex workflows
- **Production Deployment**: LangSmith tracing and monitoring setup
- **Advanced Evaluation**: Custom metrics and automated testing pipelines
- **Cost Optimization**: Token usage analysis and efficiency improvements

### 2025 Production Considerations:
- Enable LangSmith tracing for production monitoring
- Implement DeepEval for comprehensive agent testing
- Use conversation threading for memory-enabled applications
- Monitor costs with built-in callback handlers
- Leverage LangGraph for complex multi-agent orchestration